In [26]:
# No import

## Grammar
Ceci exite juste pour les tests du prototype, pas besoin de les recopiers en Ocaml, il s'agit d'une construction de l'ast de la grammaire.

In [27]:
class Z:  
    def __str__(self):
        return 'Z'
    
class B: 
    def __str__(self):
        return 'B'
    
class E:
    def __str__(self):
        return 'E'

In [28]:
class Component(list):
    
    def __init__(self,String):
        String = String.replace(" ","")
        List = String.split("*")
        for e in List:
            if e is "Z":
                self.append(Z())
            if e is "B":
                self.append(B())
            if e is "E":
                self.append(E())
            
    def __str__(self):
        s = ""
        for e in self:
            s = s + str(e) + " * "
        return s[:-3]  

In [29]:
class Rule():
    
    def __init__(self,String):
        String = String.replace(" ","")
        List = String.split("=")
        self.name,self.components = List[0],List[1].split("+")
        for i in range(0,len(self.components)):
            self.components[i] = Component(self.components[i])
        
        
        
    def __str__(self):
        s = self.name + " = "
        for component in self.components:
            s = s + str(component) + " + "
        return s[:-3]

In [30]:
class Grammar(list):
    
    def __init__(self,String):
        String = String.replace(" ","")
        if String[-1] == '\n': String = String[:-1]
        List = String.split("\n")
        for i in range(0,len(List)):
            self.append(Rule(List[i]))
        
    def __str__(self):
        s = ""
        for rule in self:
            s = s + str(rule) + '\n'
        return s[:-1]

## Utilities

In [32]:
# count_recursion : Component -> int
# retourne le nombre de B dans cpn
def count_recursions(cpn):
    return len([e for e in cpn if str(e) == 'B'])
# count_nodes : Component -> int
# retourne le nombre de Z dans cpn
def count_nodes(cpn):
    return len([e for e in cpn if str(e) == 'Z'])
# count_leafs : Component -> int
# retourne le nimbre de E dans cpn
def count_leafs(cpn):
    return len([e for e in cpn if str(e) == 'E'])

In [41]:
# permutation : List -> List
# retourne toutes les permutations possible avec les éléments
# \de List en suppriments les doublons dans le resultat
def permutation(List):
    if len(List) <= 1: return List
    result = []
    for i in range(len(List)):
        _res = permutation(List[:i] + List[i+1:]) # permutation(List - {List[i]})
        if type(_res[0]) is list:
            for e in _res:
                if not [List[i]] + e in result:
                    result.append([List[i]] + e)
        else:
            if not [List[i]] + _res in result:
                result.append([List[i]] + _res)
    return result

# enumeration : Component * int -> List
# énumère toutes les combinaisons de fils possible de poids total n pour cpn
# \sans que l'une d'entre elle soit une permutation de l'autre
def enumeration(cpn,n):
    if count_recursions(cpn) == 0: return []
    base = [0] * count_recursions(cpn)
    print("n",n)
    base[0] = n
    print(base[0])
    result = []
    while result == [] or result[-1] != base:
        result.append(base.copy())
        for i in range(len(base) - 1):
            if base[i] - 1 >= base[i+1] + 1:
                base[i] -= 1
                base[i+1] += 1
                break
    return result

# sons_possibilities : Component * int -> List of List
# retourne la listes des combinaisons de fils possible de poids total n pour cpn
def sons_possibilities(cpn,n):
    result = []
    enum = enumeration(cpn,n)
    print(enum)
    if type(enum[0]) is int: print("Coucou")
    print(type(enum[0]))
    for e in enum:
        result += permutation(e)
    print(result)
    if result != [] and type(result[0]) == int: return [result]
    return result

In [42]:
# leaf_minimum : Rule * int -> int
# retourne le poids de la feuille de poids minimum
def leaf_minimum(rule,N):
    leaf_min = N
    for cpn in rule.components:
        print(cpn)
        if count_recursions(cpn) == 0:
            cost = count_nodes(cpn)
            if cost < leaf_min:
                leaf_min = cost
    return leaf_min

In [43]:
# number_of_possibilities : List * List -> int
# retourn le produit : prod from i=0 to len(l) Bi
def number_of_possibilities(list_of_pred,l):
    result = 1
    if l == []: return result
    for e in l:
        print("e",e)
        result *= list_of_pred[e]
    return result

In [44]:
# count : int * Rule -> int
# retourne le nombre d'arbre possible de taille n qui vériefient rule
def count(N,rule): #count : int * Rule -> int
    
    leaf_min = leaf_minimum(rule,N)
    
    list_of_B = []
    
    for n in range(0,N+1):
        print("-----",n,"-----")
        Bn = 0
        for cpn in rule.components:
            print(cpn)
            print("-> before")
            cost_max = count_nodes(cpn) + count_recursions(cpn) * N
            cost_min = count_nodes(cpn) + count_recursions(cpn) * leaf_min
            print(":::",cost_min,cost_max)
            if cost_min <= n and cost_max >= n:
                print("-> first")
                if cost_min == cost_max:
                    print("-> if")
                    Bn = Bn + 1
                else:
                    print("-> else")
                    possibilities = sons_possibilities(cpn,n - count_nodes(cpn))
                    print("\tpossibilities : ",possibilities)
                    for possibility in possibilities:
                        print(possibility)
                        Bn = Bn + number_of_possibilities(list_of_B,possibility)
            print("<-")
        list_of_B.append(Bn)
        print("B" + str(n),"=",Bn)
    return list_of_B[N]

In [40]:
G = Grammar("B = E + Z + Z*B + Z*B*B")

count(5,G[0])

E
Z
Z * B
Z * B * B
----- 0 -----
E
-> before
::: 0 0
-> first
-> if
<-
Z
-> before
::: 1 1
<-
Z * B
-> before
::: 1 6
<-
Z * B * B
-> before
::: 1 11
<-
B0 = 1
----- 1 -----
E
-> before
::: 0 0
<-
Z
-> before
::: 1 1
-> first
-> if
<-
Z * B
-> before
::: 1 6
-> first
-> else
n 0
0
[[0]]
<class 'list'>
permutation : [0]
[0]
	possibilities :  [[0]]
[0]
e 0
<-
Z * B * B
-> before
::: 1 11
-> first
-> else
n 0
0
[[0, 0]]
<class 'list'>
permutation : [0, 0]
permutation : [0]
permutation : [0]
[[0, 0]]
	possibilities :  [[0, 0]]
[0, 0]
e 0
e 0
<-
B1 = 3
----- 2 -----
E
-> before
::: 0 0
<-
Z
-> before
::: 1 1
<-
Z * B
-> before
::: 1 6
-> first
-> else
n 1
1
[[1]]
<class 'list'>
permutation : [1]
[1]
	possibilities :  [[1]]
[1]
e 1
<-
Z * B * B
-> before
::: 1 11
-> first
-> else
n 1
1
[[1, 0]]
<class 'list'>
permutation : [1, 0]
permutation : [0]
permutation : [1]
[[1, 0], [0, 1]]
	possibilities :  [[1, 0], [0, 1]]
[1, 0]
e 1
e 0
[0, 1]
e 0
e 1
<-
B2 = 9
----- 3 -----
E
-> before
::: 0 0
<

783